In [3]:
import json
import re
from openpyxl import load_workbook
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.worksheet.datavalidation import DataValidation
from urllib.parse import urlparse

# === Constants ===
EXCEL_FILE = "/Users/asadeg02/Downloads/Resource Glossary AB_AT.xlsx"
SHEET_NAME = "Sheet1"
START_ROW = 3
END_ROW = 115
TAG_COLUMN_INDEX = 1    # Column A
TITLE_COLUMN_INDEX = 2  # Column B
LINK_COLUMN_INDEX = 3   # Column C

TAG_DROPDOWNS_FILE_PATH = "tag_dropdowns.json"
RESOURCE_DICT_FILE_PATH = "resource_dict.json"

# === Helpers ===
def is_google_link(link):
    return isinstance(link, str) and ('drive.google.com' in link or 'docs.google.com' in link or 'forms.gle' in link)

def extract_hyperlink(ws: Worksheet, row: int, col: int) -> str:
    cell = ws.cell(row=row, column=col)
    return cell.hyperlink.target if cell.hyperlink else cell.value

# === Core Functions ===
def parse_tag_dropdowns(sheet: Worksheet) -> dict:
    tag_dropdowns = {}

    for dv in sheet.data_validations.dataValidation:
        if dv.type == "list" and dv.formula1:
            options = dv.formula1.strip('"').split(',')
            for cell_range in dv.ranges.ranges:
                # Only consider ranges that include TAG_COLUMN_INDEX
                if TAG_COLUMN_INDEX >= cell_range.min_col and TAG_COLUMN_INDEX <= cell_range.max_col:
                    for row in range(cell_range.min_row, cell_range.max_row + 1):
                        if START_ROW <= row <= END_ROW:
                            tag_value = sheet.cell(row=row, column=TAG_COLUMN_INDEX).value
                            if tag_value:
                                tag_dropdowns[tag_value] = options
    return tag_dropdowns

def parse_resources(ws: Worksheet, tag_dropdowns: dict) -> dict:
    resource_dict = {}

    for row in range(START_ROW, END_ROW + 1):
        title = ws.cell(row=row, column=TITLE_COLUMN_INDEX).value
        link = extract_hyperlink(ws, row, LINK_COLUMN_INDEX)
        tag_label = ws.cell(row=row, column=TAG_COLUMN_INDEX).value
        
        if title and link and is_google_link(link):
            tag_options = tag_dropdowns.get(tag_label, [])            
            resource_dict[title] = {
                "link": convert_to_direct_link(link),
                "tags": tag_options
            }

    return resource_dict

def save_json(data: dict, file_path: str):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

def extract_file_id(url: str) -> str | None:
    """
    Extracts the Google file ID from Docs or Drive URLs.
    """
    # Match document, spreadsheet, presentation, or drive file
    patterns = [
        r'document/d/([a-zA-Z0-9_-]+)',
        r'spreadsheets/d/([a-zA-Z0-9_-]+)',
        r'presentation/d/([a-zA-Z0-9_-]+)',
        r'file/d/([a-zA-Z0-9_-]+)'
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def is_google_docs_link(url: str) -> bool:
    return bool(re.search(r'docs\.google\.com/(document|spreadsheets|presentation)/d/', url))

def is_google_drive_file(url: str) -> bool:
    return bool(re.search(r'drive\.google\.com/file/d/', url))

def convert_to_direct_link(url: str) -> str | None:
    """
    Converts a Google Doc/Sheet/Slide or Drive link to a direct downloadable/viewable link.
    """
    file_id = extract_file_id(url)
    if not file_id:
        return None

    if is_google_docs_link(url):
        # For Docs/Sheets/Slides, export as PDF
        if "document" in url:
            return f"https://docs.google.com/document/d/{file_id}/export?format=pdf"
        elif "spreadsheets" in url:
            return f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx"
        elif "presentation" in url:
            return f"https://docs.google.com/presentation/d/{file_id}/export/pdf"
    elif is_google_drive_file(url):
        # For Drive file (PDFs, images, zips, etc.)
        return f"https://drive.google.com/uc?export=download&id={file_id}"

    return None

# === Main Runner ===
def main():
    wb = load_workbook(EXCEL_FILE, data_only=True)
    sheet = wb[SHEET_NAME]

    tag_dropdowns = parse_tag_dropdowns(sheet)
    resources = parse_resources(sheet, tag_dropdowns)

    save_json(tag_dropdowns, TAG_DROPDOWNS_FILE_PATH)
    save_json(resources, RESOURCE_DICT_FILE_PATH)
    print(f"Saved {len(tag_dropdowns)} tag dropdown entries to '{TAG_DROPDOWNS_FILE_PATH}'")
    print(f"Saved {len(resources)} resources to '{RESOURCE_DICT_FILE_PATH}'")

if __name__ == "__main__":
    main()

Saved 19 tag dropdown entries to 'tag_dropdowns.json'
Saved 62 resources to 'resource_dict.json'


In [8]:
import json
import requests
import boto3
from urllib.parse import urlparse, parse_qs
import os

import boto3

def clear_dynamo_table(table_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)

    def get_key(item):
        return {k['AttributeName']: item[k['AttributeName']] for k in table.key_schema}

    scan = table.scan()
    with table.batch_writer() as batch:
        for item in scan['Items']:
            batch.delete_item(Key=get_key(item))

    while 'LastEvaluatedKey' in scan:
        scan = table.scan(ExclusiveStartKey=scan['LastEvaluatedKey'])
        with table.batch_writer() as batch:
            for item in scan['Items']:
                batch.delete_item(Key=get_key(item))

    print(f"All items deleted from {table_name}")

def upload_pdfs_to_s3(json_path, s3_bucket):
    s3 = boto3.client('s3')

    with open(json_path, 'r') as f:
        data = json.load(f)

    for title, info in data.items():
        link = info.get("link")
        if not link:
            print(f"Skipping (no link): {title}")
            continue

        parsed_url = urlparse(link)
        query_params = parse_qs(parsed_url.query)
        path = parsed_url.path.lower()

        # Determine extension
        file_ext = None
        if 'format' in query_params:
            file_ext = query_params['format'][0].lower()
        elif '.' in path:
            file_ext = path.split('.')[-1]
        else:
            file_ext = "unknown"

        is_pdf = file_ext == 'pdf'

        '''if not is_pdf:
            print(f"Skipping (not a PDF): {title} => Type: {file_ext} | URL: {link}")
            continue'''

        try:
            # Download file into memory
            response = requests.get(link)
            response.raise_for_status()
            filename = title.replace(" ", "_").replace("/", "_") + ".pdf"

            # Upload to S3
            s3.put_object(
                Bucket=s3_bucket,
                Key=filename,
                Body=response.content,
                ContentType='application/pdf'
            )
            print(f"Uploaded: {filename}")
        except Exception as e:
            print(f"Failed to process {title}: {e}")

clear_dynamo_table('ResourceIndex')
clear_dynamo_table('KeywordIndex')
upload_pdfs_to_s3("resource_dict.json", "career-resources")

All items deleted from ResourceIndex
All items deleted from KeywordIndex
Uploaded: A_ROADMAP_FOR_INDUSTRY_ENGAGEMENT_&_ACTION_ACROSS_SECTORS.pdf
Uploaded: Industry_and_Business_Opportunities_From_Federal_Funding_Sources_for_Supporting_Career_Readiness_and.pdf
Uploaded: BLFP_2023_Speaker_Series:_Empowering_Employers_to_Share_Career_Readiness.pdf
Uploaded: PROVIDING_QUALITY_CAREER_DEVELOPMENT_WITHIN_YOUTH_DEVELOPMENT_&_LEADERSHIP_PROGRAMS.pdf
Uploaded: Resource_Materials.pdf
Uploaded: Guideposts_for_success_2.0.pdf
Uploaded: Implementing_ILP,_How_to_Guide_V3.pdf
Uploaded: Implementing_ILP,_How_to_Guide_V2.pdf
Uploaded: ILP_-_Meaning,_Best_Practices,_Relevance_for_Stakeholders.pdf
Skipping (no link): Past Presentations by Scott on ILP/PCAP [Presentation] [Center]
Uploaded: Implementing_MyCAP.pdf
Uploaded: Best_Practices_for_Implementing_PCAP.pdf
Uploaded: Importance_of_PCAP.pdf
Uploaded: MA_Report_on_Implementing_ILP.pdf
Uploaded: Using_ILPs_-_Findings_&_Recommendations.pdf
Uploaded: NCWD